In [1]:
import os

# 设置工作路径
os.chdir('/content/sample_data')

# 检查当前工作路径
current_directory = os.getcwd()

In [3]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.5 MB/s eta 0:00:00


In [4]:
import pandas as pd
from transformers import BertTokenizer, BertModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import torch

# 载入数据
data = pd.read_csv("classification_with_GIS.csv")  # 替换为您的数据集文件路径
# 删除包含缺失值的行

In [5]:
import pandas as pd
from transformers import BertTokenizer, BertModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import torch

# 载入数据
#data = pd.read_csv("output_with_fire_prob.csv")  # 替换为您的数据集文件路径
# 删除包含缺失值的行
data = data.dropna()
# 加载BERT模型和分词器
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')


details = data['details']
labels = data['classification']

# 将文本数据转换成词向量
vectors = []
for detail in details:
    encoded = tokenizer.encode_plus(
        detail,
        add_special_tokens=True,
        max_length=128,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    with torch.no_grad():
        output = model(**encoded)[0][:, 0, :].squeeze().numpy()
        vectors.append(output)

# 训练随机森林模型
rf_model = RandomForestClassifier(n_estimators=100)
rf_model.fit(vectors, labels)
rf_preds = rf_model.predict(vectors)
rf_predictions = rf_model.predict(vectors)
# rf_accuracy = accuracy_score(labels, rf_predictions)
# print("随机森林模型准确率：", rf_accuracy)

# 训练KNN模型
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(vectors, labels)
knn_preds = knn_model.predict(vectors)

# 输出分类结果
print('Random Forest Classification Report:')
print(classification_report(labels, rf_preds))
print('KNN Classification Report:')
print(classification_report(labels, knn_preds))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Random Forest Classification Report:
              precision    recall  f1-score   support

     Class 1       0.96      0.56      0.70        45
     Class 2       0.89      0.76      0.82       171
     Class 3       0.86      0.66      0.75       147
     Class 4       0.84      0.97      0.90       508

    accuracy                           0.86       871
   macro avg       0.89      0.74      0.79       871
weighted avg       0.86      0.86      0.85       871

KNN Classification Report:
              precision    recall  f1-score   support

     Class 1       0.43      0.13      0.20        45
     Class 2       0.52      0.49      0.50       171
     Class 3       0.44      0.42      0.43       147
     Class 4       0.73      0.80      0.76       508

    accuracy                           0.64       871
   macro avg       0.53      0.46      0.47       871
weighted avg       0.62      0.64      0.63       871



In [6]:
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression


In [7]:
# 训练MLP模型
mlp_model = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=200)
mlp_model.fit(vectors, labels)
mlp_preds = mlp_model.predict(vectors)

# 训练贝叶斯模型（注意：贝叶斯模型通常用于文本分类，需要使用适当的数据预处理）
# 例如，将词向量转换为词频向量
# nb_model = MultinomialNB()
# nb_model.fit(vectors, labels)
# nb_preds = nb_model.predict(vectors)

# 训练逻辑回归模型
logistic_model = LogisticRegression()
logistic_model.fit(vectors, labels)
logistic_preds = logistic_model.predict(vectors)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [8]:
print('MLP Classification Report:')
print(classification_report(labels, mlp_preds))

# 如果使用了贝叶斯模型，也输出它的分类报告
# print('Naive Bayes Classification Report:')
# print(classification_report(labels, nb_preds))

print('Logistic Regression Classification Report:')
print(classification_report(labels, logistic_preds))


MLP Classification Report:
              precision    recall  f1-score   support

     Class 1       0.78      0.71      0.74        45
     Class 2       0.86      0.80      0.83       171
     Class 3       0.83      0.68      0.75       147
     Class 4       0.87      0.94      0.90       508

    accuracy                           0.86       871
   macro avg       0.83      0.78      0.81       871
weighted avg       0.85      0.86      0.85       871

Logistic Regression Classification Report:
              precision    recall  f1-score   support

     Class 1       0.87      0.60      0.71        45
     Class 2       0.87      0.67      0.75       171
     Class 3       0.86      0.61      0.71       147
     Class 4       0.81      0.97      0.89       508

    accuracy                           0.83       871
   macro avg       0.85      0.71      0.77       871
weighted avg       0.84      0.83      0.82       871



In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import RidgeClassifier


In [11]:
# 训练决策树模型
dt_model = DecisionTreeClassifier()
dt_model.fit(vectors, labels)
dt_preds = dt_model.predict(vectors)

# 训练岭回归模型
ridge_model = RidgeClassifier()
ridge_model.fit(vectors, labels)
ridge_preds = ridge_model.predict(vectors)


In [12]:
print('Decision Tree Classification Report:')
print(classification_report(labels, dt_preds))

print('Ridge Regression Classification Report:')
print(classification_report(labels, ridge_preds))


Decision Tree Classification Report:
              precision    recall  f1-score   support

     Class 1       0.73      0.80      0.77        45
     Class 2       0.80      0.87      0.83       171
     Class 3       0.87      0.66      0.75       147
     Class 4       0.89      0.92      0.90       508

    accuracy                           0.86       871
   macro avg       0.82      0.81      0.81       871
weighted avg       0.86      0.86      0.86       871

Ridge Regression Classification Report:
              precision    recall  f1-score   support

     Class 1       0.85      0.64      0.73        45
     Class 2       0.90      0.75      0.82       171
     Class 3       0.88      0.63      0.73       147
     Class 4       0.84      0.98      0.90       508

    accuracy                           0.86       871
   macro avg       0.87      0.75      0.80       871
weighted avg       0.86      0.86      0.85       871

